In [1]:
#Globale paramtere for niluAPI
import sys
import numpy as np
base_url = "https://api.nilu.no/stats/day/"
from_date = "2010-01-01"
to_date = "2018-12-31"
radius=20
#Oslo kordinater
latitude= 59.9139
longitude=10.7522

In [5]:
#Funksjon som bruker generell nilu API funksjon for å hente ut den rå dataen og lagre den i en json fil
def get_raw_air_quality_data_nilu_oslo(): 
    
    endpoint = f"{base_url}/{from_date}/{to_date}/{latitude}/{longitude}/{radius}"

    output_file =  "../../data/raw_data/raw_air_quality_nilu_oslo.json"

    sys.path.insert(0, '../../src')
    from niluAPI.get_data_niluAPI import fetch_raw_air_quality_data
    fetch_raw_air_quality_data(endpoint,  output_file)

get_raw_air_quality_data_nilu_oslo()

Gruppert data er lagret under ../../data/raw_data/raw_air_quality_nilu_oslo.json


In [ ]:
#Funksjon som bruker generell nilu API funksjon for å rense den rå dataen og lagre den i en json fil
def clean_raw_air_quality_data_nilu_oslo():
    raw_data_file =  "../../data/raw_data/air_quality_nilu_oslo.json"
    clean_data_file = "../../data/clean_data/clean_air_quality_nilu_oslo.json"
    col= ["Verdi_NO2", "Verdi_O3", "Verdi_SO2"]
    sys.path.insert(0, '../../src')
    from niluAPI.get_data_niluAPI import process_and_save_clean_data, remove_outliers
    
    process_and_save_clean_data(raw_data_file, clean_data_file, from_date, to_date)
clean_raw_air_quality_data_nilu_oslo()  


Gruppert data er lagret under ../../data/clean_data/clean_air_quality_nilu_oslo.json


In [4]:
def process_data():
    
    raw_data_file =  "../../data/raw_data/raw_air_quality_nilu_oslo.json"
    clean_data_file = "../../data/clean_data/clean_air_quality_nilu_oslo1.json"
    cols= ["Verdi_NO2", "Verdi_O3", "Verdi_SO2"]
    
    sys.path.insert(0, '../../src')
    from niluAPI.get_data_niluAPI import process_and_save_clean_data, remove_outliers
    # Først fjern outliers fra rådataene
    pivot_df = remove_outliers(raw_data_file, cols)

    # Hvis dataen ble lest riktig, prosesser og lagre dataen
    if pivot_df is not None:
        process_and_save_clean_data(pivot_df, clean_data_file, from_date, to_date)  
process_data()

Verdi_NO2:
Fjernet 49 outliers, som er mer enn 3 standardavvik unna gjennomsnittet
Standarsavvik: 22.22813278917124
Gjennomsnitt: 32.20910285475852
Verdi_O3:
Fjernet 6 outliers, som er mer enn 3 standardavvik unna gjennomsnittet
Standarsavvik: 16.32445977911544
Gjennomsnitt: 42.45968969964842
Verdi_SO2:
Fjernet 38 outliers, som er mer enn 3 standardavvik unna gjennomsnittet
Standarsavvik: 1.1625209238387966
Gjennomsnitt: 2.123869547276853
Gruppert data er lagret under ../../data/clean_data/clean_air_quality_nilu_oslo1.json
